LSTM-L Architecture based on https://arxiv.org/pdf/1711.07128.pdf

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

W0119 23:04:24.353208 13444 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0119 23:04:24.356200 13444 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','bed,bird,cat,dog,down,eight,five,four,go,happy,house,left,marvin,nine,no,off,on,one,right,seven,sheila,six,stop,three,tree,two,up,wow,yes,zero','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='lstm_l'
start_checkpoint=None
logging_interval=10
eval_step_interval=500
save_step_interval=1
silence_percentage=10.0
unknown_percentage=15.0
background_frequency=1
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002,0.0001,0.00002' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='10000,10000,10000,15000,5000' #Declare  the training steps for which the learning rates will be used
batch_size=256

In [5]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [6]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [7]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True)

..\..\_inputs\raw\train\audio\*\*.wav


W0119 23:06:07.013450 13444 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0119 23:06:07.023384 13444 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [8]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_data, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings, background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_data, train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_data, validation_ground_truth = (
        audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_data,validation_ground_truth

In [9]:
# def get_test_data(args):
#     '''
#     Input: (sess,offset)
#     '''
#     sess,i=args
#     test_data, test_ground_truth = audio_processor.get_data(
#         batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
#     return test_data,test_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    data_size = model_settings['data_size']
    label_count = model_settings['label_count']
    data_input = tf.placeholder(
      tf.float32, [None, data_size], name='data_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    # Create Model
    
    logits, dropout_prob = models.create_model(
      data_input,
      model_settings,
      model_architecture,
      is_training=True)
    
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,data_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [15]:
tf.app.run(main=main)

W0119 23:08:28.659476 13444 deprecation.py:323] From ..\libs\models.py:506: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0119 23:08:28.675407 13444 deprecation.py:323] From ..\libs\models.py:511: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0119 23:08:28.737274 13444 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0119 23:08:28.7

I0119 23:10:08.799263 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-40"
I0119 23:10:11.292912 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-41"
I0119 23:10:13.582367 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-42"
I0119 23:10:16.888868 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-43"
I0119 23:10:19.609978 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-44"
I0119 23:10:22.147418 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-45"
I0119 23:10:24.611724 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-46"
I0119 23:10:27.057307 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-47"
I0119 23:10:29.586436 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-48"
I0119 23:10:32.227835 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-49"
I0119 23:10:34.730355 13444 trainer.py:126] Step #50: rate 0.000500, a

I0119 23:13:22.609913 13444 trainer.py:126] Step #120: rate 0.000500, accuracy 21.5%, cross entropy 2.651625
I0119 23:13:22.610910 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-120"
I0119 23:13:24.844265 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-121"
I0119 23:13:27.139395 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-122"
I0119 23:13:29.392431 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-123"
I0119 23:13:31.665889 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-124"
I0119 23:13:33.985603 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-125"
I0119 23:13:36.267466 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-126"
I0119 23:13:38.552129 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-127"
I0119 23:13:40.941351 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-128"
I0119 23:13:43.094201 13444 trainer.py:172] S

I0119 23:16:19.727652 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-198"
I0119 23:16:21.747265 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-199"
I0119 23:16:24.096392 13444 trainer.py:126] Step #200: rate 0.000500, accuracy 28.9%, cross entropy 2.362973
I0119 23:16:24.098387 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-200"
I0119 23:16:26.217741 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-201"
I0119 23:16:28.295508 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-202"
I0119 23:16:30.728236 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-203"
I0119 23:16:32.870166 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-204"
I0119 23:16:35.310532 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-205"
I0119 23:16:37.409959 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-206"
I0119 23:16:39.496324 13444 trainer.py:172] S

I0119 23:18:55.300046 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-276"
I0119 23:18:57.257976 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-277"
I0119 23:18:59.160240 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-278"
I0119 23:19:01.090656 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-279"
I0119 23:19:03.028883 13444 trainer.py:126] Step #280: rate 0.000500, accuracy 45.7%, cross entropy 1.828648
I0119 23:19:03.029880 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-280"
I0119 23:19:04.964841 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-281"
I0119 23:19:06.886169 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-282"
I0119 23:19:08.788892 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-283"
I0119 23:19:10.640762 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-284"
I0119 23:19:12.591088 13444 trainer.py:172] S

I0119 23:21:27.828566 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-354"
I0119 23:21:29.690391 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-355"
I0119 23:21:31.397186 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-356"
I0119 23:21:33.300391 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-357"
I0119 23:21:34.962261 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-358"
I0119 23:21:36.780025 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-359"
I0119 23:21:38.559194 13444 trainer.py:126] Step #360: rate 0.000500, accuracy 53.9%, cross entropy 1.601291
I0119 23:21:38.560191 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-360"
I0119 23:21:40.555496 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-361"
I0119 23:21:42.975142 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-362"
I0119 23:21:44.996005 13444 trainer.py:172] S

I0119 23:23:49.204384 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-432"
I0119 23:23:51.032024 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-433"
I0119 23:23:52.825923 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-434"
I0119 23:23:54.558892 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-435"
I0119 23:23:56.301700 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-436"
I0119 23:23:58.061500 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-437"
I0119 23:23:59.916648 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-438"
I0119 23:24:01.675815 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-439"
I0119 23:24:03.434174 13444 trainer.py:126] Step #440: rate 0.000500, accuracy 48.8%, cross entropy 1.737206
I0119 23:24:03.434174 13444 trainer.py:172] Saving to "..\..\logs&checkpoint\lstm_l\ckpt-440"
I0119 23:24:05.255859 13444 trainer.py:172] S

KeyboardInterrupt: 

### Freeze

In [11]:
# save_checkpoint='..\\..\\logs&checkpoint\\lstm_l\\ckpt-50000'

In [13]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)

In [14]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True)